# <span style="color:blue"> B. Specific details for programmers: how it works</span>

## <span style="color:purple">Morphological analysis</span>

During the (full) morphological analysis, words will be segmented into morphemes and related linguistic units (e.g. roots/lemmas, suffixes), grammatical categories related to these morphemes (such as part of speech, form name) will be determined, and, in case of ambiguities, correct analyses for each word will be chosen based on contextual cues.

The core of morphological analysis is the VabamorfTagger. VabamorfTagger uses `'words'` and `'sentences'` as input layers, and tags the `'morph_analysis'` layer on the `'words'` layer. Morphological processing is done sentence-by-sentence, so the information about sentence boundaries is also required. 

In [1]:
from estnltk import Text
from estnltk.taggers import VabamorfTagger

# Initialize new morphological analyser
morph_tagger = VabamorfTagger()

# Prepare text
text=Text("Tõmba äpp kohe alla!")
# Tag layers required by morph analysis
text.tag_layer(['words','sentences'])

# Tag morph analysis
morph_tagger.tag( text )

# Results
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Tõmba', [{'lemma': 'tõmbama', 'root': 'tõmba', 'root_tokens': ['tõmba'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}]),
Span('äpp', [{'lemma': 'äpp', 'root': 'äpp', 'root_tokens': ['äpp'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('kohe', [{'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('alla', [{'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('!', [{'lemma': '!', 'root': '!', 'root_tokens': ['!'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Note that unlike previous layers (sentences, paragraphs), the morphological analysis layer is ambiguous. This means that one word can have more than one analysis. While VabamorfTagger also includes a disambiguation step, some linguistically difficult cases still remain ambiguous even after the disambiguation.

Under the hood, VabamorfTagger applies three processing steps:

  1. **Morphological analysis** of all the words in the input text, including guessing analyses for unknown words and proper names. This is done by applying a special tagger called `VabamorfAnalyzer`;
  
  2. **Post-processing of analyses**, which includes correcting words that contain numbers, correcting part of speech of compound tokens (such as names with initials, emoticons, abbreviations, numerics etc.), and marking some of the words as to be ignored during the morphological disambiguation; This is done by `PostMorphAnalysisTagger`;
  
  3. **Morphological disambiguation**, which involves picking out the most probable analyses for each word (based on the sentence context); This is done by a special tagger called `VabamorfDisambiguator`;

If you need to analyse morphologically non-standard varieties of Estonian (such as the Internet slang, transcripts of the spoken language, texts written in a dialect), you may want to decompose the morphological analysis process into substeps, make step-wise adaptations, and finally tie these adaptations together into a custom morphological analyser. For these purposes, we will also introduce sub-components of the morphological analyser.

### VabamorfAnalyzer

VabamorfAnalyzer generates morphological analyses for all the words in the input text. It also uses special heuristics to guess analyses for unknown words and proper names.

VabamorfAnalyzer also takes account the word normalization provided in the `'words'` layer: if word's attribute `'normalized_form'` contains a normalization for the word, then this normalization is used as the input in morphological analysis.

In [2]:
from estnltk import Text
from estnltk.taggers import VabamorfAnalyzer

# Initialize morphological analyser
morph_analyzer = VabamorfAnalyzer()

text=Text("Lae äpp kohe alla!")
# Tag layers required by morph analysis
text.tag_layer(['words','sentences'])

# Tag morph analysis
morph_analyzer.tag( text )

Text(text='Lae äpp kohe alla!')

Examining the results, we can note that the resulting layer is ambiguous, and contains guesses:

In [3]:
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Lae', [{'lemma': 'Laad', 'root': 'Laad', 'root_tokens': ['Laad'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H'}, {'lemma': 'Lae', 'root': 'Lae', 'root_tokens': ['Lae'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H'}, {'lemma': 'Lae', 'root': 'Lae', 'root_tokens': ['Lae'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}, {'lemma': 'Lagi', 'root': 'Lagi', 'root_tokens': ['Lagi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H'}, {'lemma': 'laad', 'root': 'laad', 'root_tokens': ['laad'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}, {'lemma': 'laadima', 'root': 'laadi', 'root_tokens': ['laadi'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}, {'lemma': 'lagi', 'root': 'lagi', 'root_tokens': ['lagi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('äpp', [{'lemma': 'äpp', 'root': 'äpp', 'root_tokens': ['äpp'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('kohe', [{'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}, {'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('alla', [{'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}, {'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'K'}]),
Span('!', [{'lemma': '!', 'root': '!', 'root_tokens': ['!'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Morphological categories (possible values for `partofspeech` and `form`) used in analyses are described [here](https://github.com/Filosoft/vabamorf/blob/master/doc/tagset.html) and [here](http://www.filosoft.ee/html_morf_et/morfoutinfo.html#2) (in Estonian).

#### Analysis parameters

At the initialization of VabamorfAnalyzer, you can also change the analysis parameters: set boolean values for keyword arguments `propername`, `guess`, `compound` and `phonetic`.

Parameters `guess` and `propername` can be used to switch guessing off:

In [4]:
# Remove old morph analysis layer
delattr(text, 'morph_analysis')

# Initialize morphological analyser that does not guess proper names
morph_analyzer = VabamorfAnalyzer(propername=False)

# Tag new morph analysis
morph_analyzer.tag( text )

text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Lae', [{'lemma': 'laad', 'root': 'laad', 'root_tokens': ['laad'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}, {'lemma': 'laadima', 'root': 'laadi', 'root_tokens': ['laadi'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}, {'lemma': 'lagi', 'root': 'lagi', 'root_tokens': ['lagi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('äpp', [{'lemma': 'äpp', 'root': 'äpp', 'root_tokens': ['äpp'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('kohe', [{'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}, {'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('alla', [{'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}, {'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'K'}]),
Span('!', [{'lemma': '!', 'root': '!', 'root_tokens': ['!'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

In [5]:
# Remove old morph analysis layer
delattr(text, 'morph_analysis')

# Initialize morphological analyser that does not guess unknown words nor proper names
morph_analyzer = VabamorfAnalyzer(guess=False, propername=False)

# Tag new morph analysis
morph_analyzer.tag( text )

text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Lae', [{'lemma': 'laad', 'root': 'laad', 'root_tokens': ['laad'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}, {'lemma': 'laadima', 'root': 'laadi', 'root_tokens': ['laadi'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}, {'lemma': 'lagi', 'root': 'lagi', 'root_tokens': ['lagi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('äpp', [{'lemma': None, 'root': None, 'root_tokens': None, 'ending': None, 'clitic': None, 'form': None, 'partofspeech': None}]),
Span('kohe', [{'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}, {'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('alla', [{'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}, {'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'K'}]),
Span('!', [{'lemma': None, 'root': None, 'root_tokens': None, 'ending': None, 'clitic': None, 'form': None, 'partofspeech': None}])])

After guessing has been switched off, unknown words receive analyses where all attribute values are set to `None`. It is important to keep in mind that before the text can be morphologically disambiguated, these `None`-analyses must be replaced with proper analyses, or otherwise, the disambiguator throws an exception.

### PostMorphAnalysisTagger

PostMorphAnalysisTagger provides post-corrections to morphological analyses before the disambiguation process. For instance, it fixes partofspeech of compound tokens, such as abbreviations and names with initials:

In [6]:
text=Text("Raamatu toim. J. K. Köstrimäe")
# Tag layers required by morph analysis
text.tag_layer(['words','sentences'])

# Initialize morphological analyser with the default settings
morph_analyzer = VabamorfAnalyzer()
# Tag morph analysis
morph_analyzer.tag( text )

# Examine results of analysis
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Raamatu', [{'lemma': 'Raamat', 'root': 'Raamat', 'root_tokens': ['Raamat'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H'}, {'lemma': 'raamat', 'root': 'raamat', 'root_tokens': ['raamat'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('toim.', [{'lemma': 'toim', 'root': 'toim', 'root_tokens': ['toim'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('J. K. Köstrimäe', [{'lemma': 'J. K. köstrimägi', 'root': 'J. K. _köstri_mägi', 'root_tokens': ['J. K. ', 'köstri', 'mägi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}])])

Note that because PostMorphAnalysisTagger is a `Retagger`, then we need to call the method **`retag()`** for applying the tagger:

In [7]:
# Create new postanalysis tagger
from estnltk.taggers import PostMorphAnalysisTagger

postanalysis_tagger = PostMorphAnalysisTagger()

# Rewrite morph_analysis with fixes
postanalysis_tagger.retag(text)

# Re-examine the results
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', '_ignore'), spans=SL[Span('Raamatu', [{'lemma': 'Raamat', 'root': 'Raamat', 'root_tokens': ['Raamat'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H', '_ignore': False}, {'lemma': 'raamat', 'root': 'raamat', 'root_tokens': ['raamat'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S', '_ignore': False}]),
Span('toim.', [{'lemma': 'toim', 'root': 'toim', 'root_tokens': ['toim'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'Y', '_ignore': False}]),
Span('J. K. Köstrimäe', [{'lemma': 'J. K. köstrimägi', 'root': 'J. _K. _Köstri_mägi', 'root_tokens': ['J. K. ', 'köstri', 'mägi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'H', '_ignore': False}])])

In the previous example, part of speech of the abbreviation was fixed to 'Y', and part of speech of the name with initials was fixed to 'H'.

Additionally, `PostMorphAnalysisTagger` adds a temporary attribute `'_ignore'` that tells disambiguator which words should be ignored during the disambiguation.

Upon initialization of `PostMorphAnalysisTagger`, flags can be used for turning different types of post-corrections off (by default all corrections are turned on). The following boolean flags can be used:

 * `ignore_emoticons` -- if set, all emoticons (compound tokens of type `emoticon`) will be ignored during the disambiguation -- that is, their analyses will be marked with `_ignore=True`;


 * `ignore_xml_tags` -- if set, all xml tags (compound tokens of type `xml_tag`) will be ignored during the disambiguation -- that is, their analyses will be marked with `_ignore=True`;


 * `fix_names_with_initials` -- names with initials (compound tokens of type `name_with_initial`) will have their partofspeech changed to 'H' and root strings normalized: underscores added between different parts of the name, and name start positions converted to uppercase;


 * `fix_emoticons` -- all emoticons will have their part of speech changed to 'Z';

 
 * `fix_www_addresses` -- all web addresses will have their part of speech changed to 'H';


 * `fix_email_addresses` -- all email addresses will have their part of speech changed to 'H';


 * `fix_abbreviations` -- abbreviations with postags 'S' and 'H' will have their part of speech changed to 'Y';


 * `fix_number_postags` -- number tokens and numbers with percentages will have their part of speech changed from 'Y' to 'N';
 
 
 * `fix_number_analyses_using_rules` -- number tokens will be fixed using rules loaded from a CSV file. For instance, this will correct numbers with case endings (such as _'10e krooniga'_ and _'6t krooni'_) so that they will receive proper morphological form and part of speech. The constructor takes the name of the corrections CSV file as a parameter `number_analysis_rules`, and if no name is provided, [the default set of rules](https://github.com/estnltk/estnltk/tree/c42f8ace79bb3d0517c2f39c4de38e779e74b3dd/estnltk/taggers/morph_analysis/number_fixes) is used. In the CSV file, each line represents a single rule in the format:
 
         number_regexp,number_suffix,pos,form,ending
 
   For instance, the rule:
 
         (|[2-9]|([1-9][0-9]*[02-9]))6$,t,N,sg p,t
   
   defines a fix for tokens consisting of numbers that end with `6` and the suffix `t` (excluding tokens ending with `16t`) -- such tokens will have their part of speech fixed to `N`, form fixed to `sg p` and, ending fixed to `t`;
   
   You can find the default rules from the file [number_analysis_rules.csv](https://github.com/estnltk/estnltk/tree/c42f8ace79bb3d0517c2f39c4de38e779e74b3dd/estnltk/taggers/morph_analysis/number_fixes).
 
 
 * `remove_duplicates` -- duplicate morphological analyses will be removed;

Finally: if you need to change the configuration of `PostMorphAnalysisTagger` (or create your own Tagger for post corrections), you can use VabamorfTagger's argument `postanalysis_tagger` to replace the default post corrector with your own one:

In [8]:
from estnltk.taggers import VabamorfTagger
from estnltk.taggers import PostMorphAnalysisTagger

# Create a post-corrector that does not fix part of speech of emoticons
postanalysis_tagger_2 = PostMorphAnalysisTagger( fix_emoticons=False )

# Initialize VabamorfTagger (analyzer + disambiguator) with new post-corrector
vabamorf_tagger = VabamorfTagger(postanalysis_tagger = postanalysis_tagger_2)

# Input text
text=Text("Äge värk :P")
# Tag layers required by morph analysis
text.tag_layer(['words','sentences'])

# Tag morph analysis
vabamorf_tagger.tag( text )

# Examine results
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Äge', [{'lemma': 'äge', 'root': 'äge', 'root_tokens': ['äge'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}]),
Span('värk', [{'lemma': 'värk', 'root': 'värk', 'root_tokens': ['värk'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span(':P', [{'lemma': 'P', 'root': 'P', 'root_tokens': ['P'], 'ending': '0', 'clitic': '', 'form': '?', 'partofspeech': 'Y'}])])

### VabamorfDisambiguator

VabamorfDisambiguator completes the morphological analysis, and picks out analyses that are most probable for a word in a sentence context. 

In [9]:
text=Text("Tõmba äpp kohe alla!")

# Tag layers required by morph analysis
text.tag_layer(['words','sentences'])

# Tag morph analysis
morph_analyzer.tag( text )

# Examine that the results are ambiguous
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Tõmba', [{'lemma': 'tõmbama', 'root': 'tõmba', 'root_tokens': ['tõmba'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}]),
Span('äpp', [{'lemma': 'äpp', 'root': 'äpp', 'root_tokens': ['äpp'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('kohe', [{'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}, {'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('alla', [{'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}, {'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'K'}]),
Span('!', [{'lemma': '!', 'root': '!', 'root_tokens': ['!'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

In [10]:
# Now, create a new disambiguator
from estnltk.taggers import VabamorfDisambiguator

vabamorf_disambiguator = VabamorfDisambiguator()

# disambiguate the text
vabamorf_disambiguator.retag( text )

# re-examine the results
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Tõmba', [{'lemma': 'tõmbama', 'root': 'tõmba', 'root_tokens': ['tõmba'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}]),
Span('äpp', [{'lemma': 'äpp', 'root': 'äpp', 'root_tokens': ['äpp'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('kohe', [{'lemma': 'kohe', 'root': 'kohe', 'root_tokens': ['kohe'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('alla', [{'lemma': 'alla', 'root': 'alla', 'root_tokens': ['alla'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('!', [{'lemma': '!', 'root': '!', 'root_tokens': ['!'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

If the input layer contains attribute `'_ignore'`, then words which analyses have `_ignore=True` will be completely ignored during the disambiguation. This can be used to exclude XML tags, and other similar "text formatting elements" from being analysed. 

Important note: when changing `'_ignore'` values, please keep in mind that a selective ignoring of word's analyses is currently not allowed -- either all analyses of the word should be set to `_ignore=True`, or none of them.

Also note that the attribute `'_ignore'` is temporary, and it will be removed during the disambiguation. So, the new 'morph_analysis' layer created by the disambiguator does not have the attribute.

**(!)** It is important to re-iterate: if you apply morphological analyzer with guessing switched off, and/or you remove  analyses during the post-correction, you must make sure that before applying the disambiguation, all words have morphological  analyses. If VabamorfDisambiguator stumbles upon words that do not have analyses (or have analysis with all attributes set to `None`), it raises an exception.

---

### [Advanced topic] Changing Vabamorf's binary lexicons

Under the hood, `VabamorfTagger`, `VabamorfAnalyzer` and `VabamorfDisambiguator` are using Filosoft's [Vabamorf](https://github.com/Filosoft/vabamorf) tool for performing morphological analysis and disambiguation.
The Vabamorf tool has [two binary lexicon files](https://github.com/Filosoft/vabamorf/tree/master/dct/binary): `et.dct` (analyzer's lexicon), and `et3.dct` (disambiguator's lexicon).
How to create and compile these lexicons is out of the scope of tutorial (see Vabamorf's github documentation for details). 
But once you have managed to obtain new binary lexicons, you can also use these in EstNLTK.
For this, you first need to create a new `VabamorfInstance` that loads customized lexicons instead of the default ones:
    
    from estnltk.vabamorf.morf import Vabamorf as VabamorfInstance
    # paths to new lexicons:
    new_lex_path = 'et_new.dct'
    new_disamb_lex_path = 'et3_new.dct'
    new_vm_instance = VabamorfInstance( lex_path=new_lex_path, disamb_lex_path=new_disamb_lex_path )
    
Now, if you create an instance of `VabamorfTagger`, `VabamorfAnalyzer` or `VabamorfDisambiguator`, you can use the constructor parameter `vm_instance` to override the default `VabamorfInstance`, so that custom lexicons are used in morphological analysis. For example:

    from estnltk.taggers import VabamorfTagger
    new_vm_tagger = VabamorfTagger(  vm_instance=new_vm_instance )
    # Tag text with new lexicons
    new_vm_tagger.tag( ... )

What to keep in mind when using customized lexicons:

 * If you decide to change lexicons, you should update both analyzer's lexicon and disambiguator's lexicon, and change both lexicons simultaneously. Changing only one of the lexicons is risky and may result in lessened quality of analysis/disambiguation;
